In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import train_and_eval as tae


res = tae.ex.run(
    named_configs=["configs/reference/ood_features_appnp.yaml"],
    config_updates={
        "data.dataset": "CoraML",
        # "run.reduced_training_metrics": True,
        # "training.eval_every": 10,
        # "training.stopping_patience": 5,
        # "data.split": "public",
        # "run.num_splits": 1,
        "run.num_splits": 1,
        "run.num_inits": 2,
        "run.log": False,
        # "model.model_name": "GPN_LOP",
        # "model.sparse_x_prune_threshold": 0.01,
        # "run.reeval": True,
        "run.experiment_name": "ood_features_normal",
        "data.ood_perturbation_type": "normal",
    },
    options={"--force": True},
)

res.result

WARNING - train_and_eval - No observers have been added to this run
INFO - train_and_eval - Running command 'run_experiment'
INFO - train_and_eval - Started
INFO - root - Received the following configuration:
INFO - root - RUN
INFO - root - {'experiment_name': 'ood_features_normal', 'eval_experiment_name': 'classification', 'experiment_directory': './saved_experiments', 'reduced_training_metrics': False, 'eval_mode': 'default', 'job': 'evaluate', 'save_model': False, 'gpu': 0, 'num_inits': 2, 'num_splits': 1, 'log': False, 'debug': True, 'ex_type': 'transductive', 'ood_loc': True, 'ood_loc_only': False, 'ood_edge_perturbations': True, 'ood_isolated_perturbations': False}
INFO - root - -----------------------------------------
INFO - root - DATA
INFO - root - {'to_sparse': False, 'split_no': 1, 'dataset': 'CoraML', 'root': './data', 'split': 'random', 'train_samples_per_class': 0.05, 'val_samples_per_class': 0.15, 'test_samples_per_class': 0.8, 'ood_flag': True, 'ood_setting': 'evasion'

Starting experiment (model=APPNP, dataset=CoraML, ood_type=perturb_features, split=1, init=1, results=./saved_experiments/classification/APPNP/1/results_ood_features_normal_1.json, trained=True, evaluated=True).
Completed experiment (model=APPNP, dataset=CoraML, ood_type=perturb_features, split=1, init=1, results=./saved_experiments/classification/APPNP/1/results_ood_features_normal_1.json, trained=True, evaluated=True).
Starting experiment (model=APPNP, dataset=CoraML, ood_type=perturb_features, split=1, init=2, results=./saved_experiments/classification/APPNP/1/results_ood_features_normal_2.json, trained=True, evaluated=True).
Completed experiment (model=APPNP, dataset=CoraML, ood_type=perturb_features, split=1, init=2, results=./saved_experiments/classification/APPNP/1/results_ood_features_normal_2.json, trained=True, evaluated=True).


<string>:1: RuntimeWarning: invalid value encountered in subtract
/home/clemens/Projects/Graph-Posterior-Network/.venv/lib/python3.10/site-packages/numpy/core/_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/clemens/Projects/Graph-Posterior-Network/.venv/lib/python3.10/site-packages/numpy/core/_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
INFO - train_and_eval - Result: {'test_accuracy': 0.3930785059928894, 'test_brier_score': 0.8468131422996521, 'test_ECE': 0.44350095093250275, 'test_confidence_aleatoric_apr': 0.39635269144963103, 'test_confidence_epistemic_apr': nan, 'test_confidence_structure_apr': nan, 'test_confidence_aleatoric_auroc': 0.49808597542355393, 'test_confidence_epistemic_auroc': nan, 'test_confidence_structure_auroc': nan, 'test_CE': inf, 'test_avg_prediction_confidence_aleatoric': 0.8343484997749329, 'test_avg_prediction_confidence_epistemic': nan, 'tes

{'test_accuracy': 0.3930785059928894,
 'test_brier_score': 0.8468131422996521,
 'test_ECE': 0.44350095093250275,
 'test_confidence_aleatoric_apr': 0.39635269144963103,
 'test_confidence_epistemic_apr': nan,
 'test_confidence_structure_apr': nan,
 'test_confidence_aleatoric_auroc': 0.49808597542355393,
 'test_confidence_epistemic_auroc': nan,
 'test_confidence_structure_auroc': nan,
 'test_CE': inf,
 'test_avg_prediction_confidence_aleatoric': 0.8343484997749329,
 'test_avg_prediction_confidence_epistemic': nan,
 'test_avg_sample_confidence_aleatoric': 0.8343484997749329,
 'test_avg_sample_confidence_aleatoric_entropy': 0.8343484997749329,
 'test_avg_sample_confidence_epistemic': nan,
 'test_avg_sample_confidence_epistemic_entropy': nan,
 'test_avg_sample_confidence_features': nan,
 'test_avg_sample_confidence_neighborhood': nan,
 'test_average_entropy': 0.45442643761634827,
 'test_accuracy_rejection_prediction_confidence_aleatoric': array([0.39307851, 0.3943662 , 0.39541622, 0.39589989

## Manual

In [2]:
import torch
import torch_geometric as pyg
import torch_geometric.datasets as pygd
import warnings
from sklearn.exceptions import UndefinedMetricWarning

warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

In [ ]:
from gpn.experiments.multiple_run_experiment import MultipleRunExperiment
from gpn.experiments.transductive_experiment import TransductiveExperiment
from gpn.utils.config import (
    DataConfiguration,
    ModelConfiguration,
    RunConfiguration,
    TrainingConfiguration,
)


def create_experiment(model_name):
    run_cfg = RunConfiguration(
        job="train",
        eval_mode="default",
        experiment_directory="./.cache",
        save_model=True,
        gpu=0,
        experiment_name="ood_loc",
        num_inits=1,
        num_splits=1,
    )
    data_cfg = DataConfiguration(
        dataset="CoraML",
        split_no=1,
        root="./data",
        ood_flag=False,
        train_samples_per_class=0.05,
        val_samples_per_class=0.15,
        test_samples_per_class=0.8,
        split="random",
        ood_setting="poisoning",
        ood_type="leave_out_classes",
        ood_num_left_out_classes=-1,
        ood_leave_out_last_classes=True,
    )

    model_cfg = ModelConfiguration(
        model_name=model_name,
        seed=42,
        init_no=1,
        dim_hidden=64,
        dropout_prob=0.5,
        K=10,
        add_self_loops=True,
        maf_layers=0,
        gaussian_layers=0,
        use_batched_flow=True,
        loss_reduction="sum",
        approximate_reg=True,
        flow_weight_decay=0.0,
        pre_train_mode="flow",
        alpha_evidence_scale="latent-new",
        alpha_teleport=0.1,
        entropy_reg=0.0001,
        dim_latent=16,
        radial_layers=10,
        sparse_propagation=True,
        sparse_x_prune_threshold=0.01
    )

    train_cfg = TrainingConfiguration(
        epochs=100000,
        stopping_mode="default",
        stopping_patience=50,
        stopping_restore_best=True,
        stopping_metric="val_CE",
        stopping_minimize=True,
        finetune_epochs=0,
        warmup_epochs=5,
        lr=0.01,
        weight_decay=0.001,
    )

    return MultipleRunExperiment(run_cfg, data_cfg, model_cfg, train_cfg)


gpn_e = create_experiment("GPN")
lop_e = create_experiment("GPN_LOP")
res_lop = lop_e.run()
res = gpn_e.run()
res_lop

In [ ]:
import pandas as pd
from gpn.utils.utils import results_dict_to_df

df_lop = results_dict_to_df(res_lop)
df = results_dict_to_df(res)

pd.set_option("display.max_rows", None)
(pd.DataFrame(
    dict(
        val_gpn=df["val"],
        test_gpn=df["test"],
        val_lop=df_lop["val"],
        test_lop=df_lop["test"],
    )
)).round(4).dropna()

In [ ]:
from gpn.data.dataset_manager import DatasetManager
from gpn.data.dataset_provider import InMemoryDatasetProvider
import torch_sparse as ts

ds = InMemoryDatasetProvider(
    DatasetManager(
        dataset="AmazonPhotos",
        split_no=1,
        root="./data",
        ood_flag=True,
        train_samples_per_class=0.05,
        val_samples_per_class=0.15,
        test_samples_per_class=0.8,
        split="random",
        ood_setting="poisoning",
        ood_type="leave_out_classes",
        ood_num_left_out_classes=-1,
        ood_leave_out_last_classes=True,
    )
).to_sparse()

batch = list(ds.loader())[0]

batch.adj_t.dtype()

In [ ]:
N = batch.x.shape[0]
adj_t = batch.adj_t.cuda()
adj_t

In [ ]:
import torch_sparse as ts

adj_t = ts.SparseTensor.from_dense(
    torch.Tensor([[0, 1, 0], [1, 0, 1], [0, 1, 0]])
).cuda()
x = torch.Tensor([1, 10, 100]).view(-1, 1).cuda()
N = x.shape[0]
adj_t /= x
print(adj_t.dtype())
adj_t.set_diag(torch.tensor([6,2,3], dtype=torch.float32).cuda()).to_dense()

In [ ]:
from gpn.layers.appnp_propagation import APPNPPropagation

prop = APPNPPropagation(1, 0, 0, normalization="sym")
x = ts.SparseTensor.eye(N, dtype=torch.float32, device="cuda")  # type: ignore

p: ts.SparseTensor = prop(x, adj_t)

p.to_dense()

In [ ]:
prop = APPNPPropagation(1, 0, 0, normalization="sym")
x = torch.diag(torch.ones(N, device="cuda"))

p: ts.SparseTensor = prop(x, adj_t)

p

In [ ]:
ts.SparseTensor.from_edge_index(torch.tensor([[0,1],[0,1]]),sparse_sizes=(10,3)).dtype()

In [ ]:
ts.SparseTensor.from_.get_diag()